In [6]:
import pandas as pd
import numpy as np
import aggregation
import cleaning

In [7]:
# open elections_with_tracts.xlsx
elections_with_tracts = pd.read_excel('elections_with_tracts.xlsx')
# open acs_data.csv
acs_tract_data = pd.read_csv('tract_acs_data.csv')
# open statistics.xlsx
statistics = pd.read_excel('statistics.xlsx')

/var/folders/52/t8__xqw525g96s6z30kmvnfr0000gp/T/ipykernel_65013/3175199637.py:4: DtypeWarning: Columns (13,68,69,71,72,79,105,106,122,133,135,151,171,204,208,209,218,221,224,233,238,256,262,283,287,302,320,322,334,355,361,369,370,378,394,403,419,420,421,446,447,451,457,460,464,470,475,476,505,525,532,533,539,542,551,564,574,598,607,618,631,635,648,656,657,660,673,677,684,692,693,694,703,704,729,743,756) have mixed types. Specify dtype option on import or set low_memory=False.
  acs_tract_data = pd.read_csv('tract_acs_data.csv')


In [17]:
# set elections_with_tracts to just the row where filename is SanLeandro_11062018_CountyCouncilDistrict1.csv
elections_with_tracts = elections_with_tracts[elections_with_tracts['filename'] == 'SanLeandro_11062018_CountyCouncilDistrict1.csv']

In [18]:
# replace all (X), -, or N with nan
cleaning.replace_error_values(acs_tract_data)
cleaning.combine_percent_columns(acs_tract_data)
cleaning.remove_invalid_columns(acs_tract_data)
cleaning.remove_nonaggregatables(acs_tract_data)
acs_tract_data

,geocode,year,DP03_0001E,DP02_0127E,DP03_0036E,DP02_0107E,DP05_0091E,DP02_0088E,DP02_0095E,DP03_0031E,...,DP05_0042E,DP02_0063E,DP02_0134E,DP02_0131E,DP03_0009PE,DP03_0042E,DP05_0066E,DP03_0064E,DP02_0116E,DP02_0087E
0,06_001_422200,2010,2701.0,80.0,35.0,13.0,NaN,2180.0,723.0,15.0,...,7.0,148.0,377.0,268.0,9.5,452.0,489.0,1217.0,272.0,2196.0
1,06_001_422400,2010,4531.0,74.0,0.0,17.0,NaN,2642.0,1370.0,51.0,...,59.0,63.0,235.0,252.0,1.9,1421.0,276.0,1819.0,764.0,2709.0
2,06_001_422300,2010,3045.0,86.0,9.0,0.0,NaN,3005.0,375.0,111.0,...,12.0,69.0,439.0,310.0,4.5,835.0,357.0,1261.0,194.0,3075.0
3,06_001_420400,2010,2292.0,0.0,11.0,16.0,NaN,1307.0,1943.0,0.0,...,8.0,59.0,40.0,39.0,2.9,797.0,585.0,956.0,344.0,1387.0
4,06_001_421800,2010,1809.0,31.0,0.0,66.0,NaN,1923.0,99.0,74.0,...,0.0,38.0,153.0,259.0,6.7,297.0,317.0,723.0,96.0,1944.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6405,50_007_002304,2023,2270.0,0.0,37.0,158.0,1031.0,2814.0,396.0,104.0,...,0.0,257.0,10.0,277.0,2.7,623.0,2659.0,1046.0,26.0,0.0
6406,50_007_003304,2023,5342.0,0.0,122.0,411.0,2512.0,6332.0,762.0,161.0,...,0.0,408.0,101.0,401.0,4.8,1240.0,5621.0,2358.0,181.0,21.0
6407,50_007_000100,2023,3847.0,27.0,49.0,110.0,1964.0,4508.0,640.0,263.0,...,0.0,549.0,31.0,450.0,4.6,869.0,3917.0,1533.0,11.0,0.0
6408,50_007_000800,2023,2049.0,15.0,0.0,10.0,1089.0,2430.0,99.0,97.0,...,0.0,179.0,0.0,194.0,1.2,582.0,2404.0,928.0,57.0,0.0


In [19]:
# open elections.xlsx
elections = pd.read_excel('elections.xlsx')

In [21]:
aggregated_data = []

# create df called missing_data with columns filename, year, values, num_missing, num_total, missing_tracts
missing_data = pd.DataFrame(columns=['filename', 'year', 'variable', 'values', 'num_missing', 'num_total', 'prop_missing'])

for index, row in elections_with_tracts.iterrows():
    if index % 10 == 0:
        print(f'{index} of {len(elections_with_tracts)} rows processed')

    tracts = row['tracts'].split(', ')
    year = elections[elections['filename'] == row['filename']]['year'].values[0]
    
    # get rows where geocode is in tracts and year = year and remove 0 values
    percent_dict = {tract: percent for tract, percent in zip(tracts, list(map(float, row['percents'].split(', '))))}
    tract_rows = acs_tract_data[(acs_tract_data['geocode'].isin(tracts)) & (acs_tract_data['year'] == year) & acs_tract_data['DP05_0001E'] > 0]
    percents = pd.DataFrame({'percent': [percent_dict[geocode] for geocode in tract_rows['geocode']]})
    tract_rows = pd.concat([tract_rows.reset_index(drop=True), percents], axis=1)
    tract_rows = tract_rows[tract_rows['percent'] > 0]

    election_data = {}

    for column in acs_tract_data.columns[2:]:
        # get the values in tract_rows for column
        values = tract_rows[column].values
        valid_values = values[~np.isnan(values)]

        # if all values are NaN, continue
        if all(np.isnan(values)):
            election_data[column] = np.nan
        else:
            # if at least one but not all values are NaN, add to missing data
            if any(np.isnan(values)):
                missing_data.loc[len(missing_data)] = [row['filename'], year, column, values, sum(np.isnan(values)), len(values), sum(np.isnan(values)) / len(values)]

            if column in statistics['agg_var'].values:
                pop_var = statistics[statistics['agg_var'] == column]['pop_var'].values[0]
                # if pop_var is none skip
                if pd.isnull(pop_var):
                    continue

                # if pop_var contains -
                if '-' in pop_var:
                    # split on -
                    pop_vars = pop_var.split('-')
                    pops = tract_rows[pop_vars[0]].values - tract_rows[pop_vars[1]].values
                else:
                    pops = tract_rows[pop_var].values
                
                percents = tract_rows['percent'].values
                tract_pops = pops*percents
                valid_tract_pops = tract_pops[~np.isnan(values)]

                # if sum valid_tract_pops is 0
                if sum(valid_tract_pops) == 0:
                    continue

                agg_type = statistics[statistics['agg_var'] == column]['type'].values[0]
                election_data[column] = aggregation.aggregate_statistic(agg_type, valid_values, valid_tract_pops)
            else:
                # if every value is not nan
                if any(~np.isnan(values)):
                    pops = tract_rows['DP05_0001E'].values
                    percents = tract_rows['percent'].values
                    tract_pops = pops*percents
                    # get the values in values that are not NaN as well as the corresponding values in DP05_0001E and the corresponding percents
                    valid_pops = pops[~np.isnan(values)]
                    valid_tract_pops = tract_pops[~np.isnan(values)]

                    props = valid_values / valid_pops
                    weighted_prop = aggregation.weighted_amean(props, valid_tract_pops)

                    # replace the Nan values in values with tract_pop * weighted_prop
                    for i in range(len(values)):
                        if np.isnan(values[i]):
                            values[i] = pops[i] * weighted_prop

                election_data[column] = aggregation.weighted_sum(values, percents)

    aggregated_data.append(election_data)

# save missing_data to csv
# missing_data.to_csv('missing_data.csv', index=False)

In [14]:
aggregated_df = pd.DataFrame(aggregated_data)
aggregated_df = pd.concat([elections_with_tracts['filename'], aggregated_df], axis=1)
aggregated_df

,filename,DP03_0001E,DP02_0127E,DP03_0036E,DP02_0107E,DP05_0091E,DP02_0088E,DP02_0095E,DP03_0031E,DP02_0093E,...,DP05_0042E,DP02_0063E,DP02_0134E,DP02_0131E,DP03_0009PE,DP03_0042E,DP05_0066E,DP03_0064E,DP02_0116E,DP02_0087E
0,Berkeley_11022010_CityCouncilDistrict1.csv,10550.244634,232.161021,117.025409,61.013283,NaN,9593.852007,1456.299505,257.888604,2548.302005,...,93.860840,406.249962,1281.640671,1276.021324,7.445267,2384.539956,1559.166798,4642.477391,716.389786,9838.321744
1,Berkeley_11042014_CityCouncilDistrict1.csv,12149.684370,172.520484,68.020697,52.325055,NaN,11517.205112,1349.525956,233.283793,2569.956015,...,82.836240,295.369227,1387.387008,1848.713653,6.116079,2653.553564,1465.770059,5472.625668,763.885911,11824.131525
2,Berkeley_11062018_CityCouncilDistrict1.csv,12548.617597,74.301575,108.639334,1379.785930,NaN,14921.653665,2536.160536,493.051663,270.031879,...,0.024478,1393.369306,97.933612,316.060928,4.000052,2400.757121,193.363980,5223.524492,1255.344100,242.559338
3,Minneapolis_11022021_CityCouncilWard1.csv,29288.723516,312.886798,541.942825,1281.256122,12599.857143,34001.820047,4367.040098,2167.654346,350.796561,...,15.000000,4335.699247,138.357805,959.016102,4.704190,5560.536595,26651.212055,12542.565872,1990.883129,102.537828
4,Minneapolis_11052013_Ward13CityCouncil.csv,28922.962495,448.178776,621.686734,31.999092,NaN,29192.525313,3158.060482,1855.462203,5084.818508,...,67.101888,2107.825486,4142.934852,8760.017540,4.815993,5292.857620,3483.423911,12120.271732,604.020135,29433.248467
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
351,Alaska_11082022_SenateDistrictP.csv,20427.720445,89.033040,159.178994,1019.025798,8998.326372,25140.123931,1730.873970,1309.874414,793.607665,...,11.067272,4753.315645,19.920950,428.849384,7.442783,2799.302787,18678.677376,8493.632578,1028.735393,156.709748
352,Alaska_11082022_SenateDistrictQ.csv,27918.030419,305.472486,255.876583,616.899481,11489.703951,36207.765620,1521.347733,2033.073383,979.422774,...,8.930378,6259.411494,58.024024,609.764119,5.213186,3958.777237,31921.585183,10512.644090,634.288602,213.147462
353,Alaska_11082022_SenateDistrictR.csv,38098.636676,346.670457,244.190476,764.621802,15809.950286,49478.863057,2583.303305,2110.754868,1842.946720,...,0.002352,8108.149880,18.550638,1239.169052,5.525200,5467.191159,37511.704218,14742.334252,1770.463472,837.002560
354,Alaska_11082022_SenateDistrictS.csv,21096.684117,32.354875,90.892105,1376.545398,8479.329442,29281.103770,2310.470977,2484.576096,215.900995,...,0.764924,3392.034494,2.448476,132.450624,12.422267,2621.971224,5593.916269,6445.499240,597.995540,236.128324


In [15]:
# open place_acs_data.csv
place_acs_data = pd.read_csv('place_acs_data.csv')

In [16]:
# concat place_acs_data and aggregated_df
elections_data = pd.concat([place_acs_data, aggregated_df], axis=0)
# save as final_data.csv
# elections_data.to_csv('final_data.csv', index=False)